In [1]:
using Pkg
Pkg.activate("../STLCG.jl/.")

 Activating environment at `~/projects/AutonomousRiskFramework/STLCG.jl/Project.toml`


In [2]:
using Revise
using STLCG
using EllipsisNotation


┌ Info: Precompiling STLCG [6ababdf7-4701-4f28-b3b2-5630f64e3c98]
└ @ Base loading.jl:1278


## Short demo

In [3]:
scale = -1
dims = 1
keepdims = true
distributed = true
pscale = 1

1

In [4]:
θ = Negation(LessThan(:x, 4.0))
ϕ = LessThan(:x, 0.0)
ψ = GreaterThan(:y, 0.0)
Φ = Or(subformula1=ϕ, subformula2=ψ)
op = Always(subformula=Φ, interval=[2, 10])

Always
  subformula: Or
  interval: Array{Int64}((2,)) [2, 10]
  _interval: Array{Int64}((2,)) [2, 10]
  rnn_dim: Int8 10
  steps: Int8 5
  operation: Minish (function of type typeof(Minish))


In [309]:
bs = 1
t = 25
x_dim = 1
dim = 2
x = randn(t, x_dim);
y = randn(t, x_dim);

x0 = x[:,1]
y0 = x[:,1]

formula = Then(ϕ, ψ, [2,5])
trace = (x,y)
trace0 = (x0, y0)

([-0.5934856848501052, -0.44672897840461556, -0.9467354370496377, -1.283678115206663, 1.8493096410404137, 0.07436781000789666, -1.4573501458263747, -0.32046107625683123, -0.6614793708099782, 1.354720287282051  …  0.24584306550849971, 0.00024159725350597966, 0.3027284151249807, -1.5342123108240857, 0.14461868393713737, 0.4333709713134625, 0.14884276341803657, -0.04621315226275659, 0.30336648833655394, -0.5059028690648278], [-0.5934856848501052, -0.44672897840461556, -0.9467354370496377, -1.283678115206663, 1.8493096410404137, 0.07436781000789666, -1.4573501458263747, -0.32046107625683123, -0.6614793708099782, 1.354720287282051  …  0.24584306550849971, 0.00024159725350597966, 0.3027284151249807, -1.5342123108240857, 0.14461868393713737, 0.4333709713134625, 0.14884276341803657, -0.04621315226275659, 0.30336648833655394, -0.5059028690648278])

In [310]:
formula(trace0)

25-element Array{Float64,1}:
 -1.0e6
 -1.0e6
 -1.0e6
 -1.0e6
 -1.0e6
 -0.44672897840461556
  1.4573501458263747
  1.4573501458263747
  1.4573501458263747
  1.4573501458263747
  0.07436781000789666
 -0.07619701636091504
  0.19111914883734588
  0.19111914883734588
  0.19111914883734588
  0.19111914883734588
  0.07619701636091504
 -0.00024159725350597966
  1.5342123108240857
  1.5342123108240857
  0.3027284151249807
  0.3027284151249807
  0.3027284151249807
  0.04621315226275659
  0.4333709713134625

In [303]:
trace1 = formula.subformula1(trace0[1])
trace2 = formula.subformula2(trace0[2])
Alw = Always(subformula=GreaterThan(:z, 0.0), interval=nothing)
LHS0 = permutedims(repeat(reshape(trace2, (size(trace2)..., 1)), ones(Int, length(size(trace2)))...,size(trace2)[1]), collect((length(size(trace2)) + 1):-1:1)) 
a = Int(formula.interval[1])
RHS = (ones(size(trace1))[1:a,..] * -LARGE_NUMBER, )
add_dim = length(size(trace1)) + 1

for i in a+1:size(trace2)[1]
    A = trace2[1:i-a,:]
    relevant = trace1[1:i,..]
    B = Alw(relevant[end:-1:1,..]; pscale, scale, keepdims, distributed)[end:-1:a+1,..]
    Minish(cat(A, B, dims=4); dims=4, scale, keepdims=false, distributed)
    RHS = (RHS..., Maxish(Minish(cat(A, B, dims=add_dim); dims=add_dim, scale, keepdims=false, distributed); dims=1, scale, keepdims, distributed))
end
cat(RHS..., dims=1)

25-element Array{Float64,1}:
 -1.0e6
 -1.0e6
 -2.205484205298316
 -2.205484205298316
 -0.6312488555526407
 -0.28462754833834314
 -0.28462754833834314
 -0.28462754833834314
 -0.28462754833834314
 -0.28462754833834314
 -0.28462754833834314
 -0.25912206247552355
 -0.09967732741752064
 -0.09967732741752064
 -0.921365816259977
 -1.2654622726854328
 -1.2654622726854328
 -1.2654622726854328
 -1.2400046943015357
 -1.2400046943015357
 -1.2400046943015357
 -1.3322708458473582
 -1.3322708458473582
 -1.3322708458473582
 -1.3322708458473582

In [288]:
RHS0

([-1.0e6, -1.0e6, -1.0e6, -1.0e6, -1.0e6], [-0.4571807577784622], [-0.4571807577784622], [-0.4571807577784622], [-0.4571807577784622], [-0.4571807577784622], [-0.4571807577784622], [-0.4571807577784622], [-0.4571807577784622], [-0.4571807577784622], [-0.4571807577784622], [-0.4571807577784622], [-0.4571807577784622], [-0.4571807577784622], [-0.4571807577784622], [-0.4571807577784622], [-0.4571807577784622], [-0.4571807577784622], [-0.4571807577784622], [-0.4571807577784622], [-0.4571807577784622])

In [276]:
Maxish(Minish(cat(A, B, dims=add_dim); dims=add_dim, scale, keepdims=false, distributed); dims=1, scale, keepdims, distributed))
#     break

4-element Array{Float64,1}:
 -0.4571807577784622
 -0.4571807577784622
 -0.4571807577784622
 -0.4571807577784622

In [281]:
A

4×1 Array{Float64,2}:
 -1.1209712974963366
 -0.0909651108477826
 -0.019880731035336784
 -0.45805343222137135

In [207]:
LARGE_NUMBER = 1E6
trace1 = formula.subformula1(trace[1])
trace2 = formula.subformula2(trace[2])
Alw = Always(subformula=GreaterThan(:z, 0.0), interval=nothing)
LHS = permutedims(repeat(reshape(trace2, (size(trace2)..., 1)), 1,1,1,size(trace2)[2]), [1, 4, 3, 2])
RHS = (Alw(trace1; pscale, scale, keepdims, distributed), )
        a = formula.interval[1]
        b = formula.interval[2]
        RHS = (ones(size(trace1))[:,1:b,..] * -LARGE_NUMBER, )
        N = size(trace1)[2]
        for i in b+1:size(trace2)[2]
            A = trace2[:,i-b:i-a,:]
            relevant = trace1[:,1:i,..]
            B = Alw(relevant[:,end:-1:1,:]; pscale, scale, keepdims, distributed)[:,b+1:-1:a+1,:]
            RHS = (RHS..., Maxish(Minish(cat(A, B, dims=4); dims=4, scale, keepdims=false, distributed); dims=2, scale, keepdims, distributed))
        end
        return cat(RHS..., dims=2);

([-1.0178721419703172 0.5759844433859311 … -0.2580588389139336 -1.0255215099612216],)

In [208]:
RHS[1]

1×15×1 Array{Float64,3}:
[:, :, 1] =
 -1.01787  0.575984  -0.959576  0.598861  …  0.083803  -0.258059  -1.02552

In [193]:
RHS0

15×15 Array{Float64,2}:
 -1.01787  -1.0e6     -1.0e6     -1.0e6     …  -1.0e6     -1.0e6     -1.0e6
 -1.01787   0.575984  -1.0e6     -1.0e6        -1.0e6     -1.0e6     -1.0e6
 -1.01787  -0.959576  -0.959576  -1.0e6        -1.0e6     -1.0e6     -1.0e6
 -1.01787  -0.959576  -0.959576   0.598861     -1.0e6     -1.0e6     -1.0e6
 -1.01787  -0.959576  -0.959576  -0.885839     -1.0e6     -1.0e6     -1.0e6
 -1.01787  -0.959576  -0.959576  -0.885839  …  -1.0e6     -1.0e6     -1.0e6
 -1.81831  -1.81831   -1.81831   -1.81831      -1.0e6     -1.0e6     -1.0e6
 -1.81831  -1.81831   -1.81831   -1.81831      -1.0e6     -1.0e6     -1.0e6
 -1.81831  -1.81831   -1.81831   -1.81831      -1.0e6     -1.0e6     -1.0e6
 -1.81831  -1.81831   -1.81831   -1.81831      -1.0e6     -1.0e6     -1.0e6
 -1.81831  -1.81831   -1.81831   -1.81831   …  -1.0e6     -1.0e6     -1.0e6
 -1.81831  -1.81831   -1.81831   -1.81831      -1.0e6     -1.0e6     -1.0e6
 -1.81831  -1.81831   -1.81831   -1.81831       0.083803  -1.0e6

In [194]:
RHS[1,:,1,:]

15×15 Array{Float64,2}:
 -1.01787   -1.0e6     -1.0e6     …  -1.0e6     -1.0e6     -1.0e6
  0.575984   0.575984  -1.0e6        -1.0e6     -1.0e6     -1.0e6
 -0.959576  -0.959576  -0.959576     -1.0e6     -1.0e6     -1.0e6
  0.598861   0.598861   0.598861     -1.0e6     -1.0e6     -1.0e6
 -0.885839  -0.885839  -0.885839     -1.0e6     -1.0e6     -1.0e6
 -0.569323  -0.569323  -0.569323  …  -1.0e6     -1.0e6     -1.0e6
 -1.81831   -1.81831   -1.81831      -1.0e6     -1.0e6     -1.0e6
  0.309607   0.309607   0.309607     -1.0e6     -1.0e6     -1.0e6
 -0.904906  -0.904906  -0.904906     -1.0e6     -1.0e6     -1.0e6
  1.14361    1.14361    1.14361      -1.0e6     -1.0e6     -1.0e6
  0.666715   0.666715   0.666715  …  -1.0e6     -1.0e6     -1.0e6
 -1.04651   -1.04651   -1.04651      -1.0e6     -1.0e6     -1.0e6
  0.083803   0.083803   0.083803      0.083803  -1.0e6     -1.0e6
 -0.258059  -0.258059  -0.258059     -0.258059  -0.258059  -1.0e6
 -1.02552   -1.02552   -1.02552      -1.02552   -1.0

In [304]:
formula(trace0)

LoadError: BoundsError: attempt to access (25,)
  at index [2]

In [338]:
# setting up the signals
bs = 1
t = 15
x_dim = 3
dim = 2
x = randn(t);
y = randn(t);


# rnn_cell(op, xs[1], op.h0; scale, distributed)
# STLCG.gradient(x -> sum(STLCG.rnn_cell(op, x, h0; scale, distributed)[1]), xs[1]) # <---- doesn't work
# STLCG.gradient(x -> sum(STLCG.rnn_cell(op, x, h0; scale, distributed)[2][1][2]), xs[1]) # <--- works

15-element Array{Float64,1}:
 -0.9032297660801328
  1.5880288189724607
  1.3776656410923007
  0.7909655032871441
 -0.7343591494389593
  0.23022609248013828
 -0.2713320417588114
 -1.5996628150105012
 -0.013749525065730723
  0.048680901648265815
 -0.28330567469906043
  2.025543434660606
 -0.21040245212203904
  1.266357631002014
 -1.6162762772289945

In [328]:
formula = Then(subformula1=ϕ, subformula2=ψ, interval=[2, Inf])
trace = (x,y);
ρt(formula, trace)
vcat(STLCG.gradient(x -> sum(ρ(formula, x)), trace)[1]...)

LoadError: StackOverflowError:

### Make some formulas

In [329]:
θ = Negation(LessThan(:x, 4.0))
ϕ = LessThan(:x, 0.0)
ψ = GreaterThan(:y, 0.0)
Φ = Or(subformula1=ϕ, subformula2=ψ)
op = Always(subformula=Φ, interval=[2, 10])

Always
  subformula: Or
  interval: Array{Int64}((2,)) [2, 10]
  _interval: Array{Int64}((2,)) [2, 10]
  rnn_dim: Int8 10
  steps: Int8 5
  operation: Minish (function of type typeof(Minish))


In [330]:
# parameters
pscale=1    # scale for LessThan, GreaterThan, Equal
scale=0     # scale for the minish and maxish function used in temporal operators, implies, and, or
keepdims=true      # keep original dimension (should pretty much always be true)
distributed=false  # if there are multiple indices that have the same max/min values, then mean over those to the gradient flows through all those values

false

### Evaluating robustness trace

In [333]:
# there are two ways to do this
ϕ(x; pscale, scale, keepdims, distributed)
STLCG.ρt(ϕ, x; pscale, scale, keepdims, distributed)

15-element Array{Float64,1}:
  0.21509389667884596
  0.2511643494757432
  1.1377041877668164
 -1.0822034242734682
  0.8395803892493985
 -0.11818520867797115
 -0.3282606051688034
 -0.06300353272339833
  0.3816041701927383
 -1.5559439019185974
 -0.05256065162916178
  1.5431426389720044
  0.9076708017146328
  0.20065892589325995
 -0.05664896029749181

### Evaluating robustness $\rho$ (not trace)

In [334]:
# there are three ways to do this
# ρ(ϕ, x; pscale, scale, keepdims, distributed)
STLCG.ρ(ϕ, x; pscale, scale, keepdims, distributed)
ϕ(x; pscale, scale, keepdims, distributed)[:,end:end,..];


In [335]:
□ϕ = STLCG.□(op, interval=[2,4])

Always
  subformula: Always
  interval: Array{Int64}((2,)) [2, 4]
  _interval: Array{Int64}((2,)) [2, 4]
  rnn_dim: Int8 4
  steps: Int8 5
  operation: Minish (function of type typeof(Minish))


In [336]:
robustness_trace(□ϕ, x; args...)

LoadError: UndefVarError: args not defined

In [ ]:
STLCG.gradient(x -> sum(STLCG.rnn_cell(op, x, h0; scale, distributed)[2][1][2]), xs[1]) # <--- works

In [4]:
ϕ = STLCG.LessThan(:x, 5)
ψ = STLCG.GreaterThan(:x, -3)
op = STLCG.Or(subformula1=STLCG.And(subformula1=ϕ, subformula2=ψ), subformula2=ϕ)

STLCG.Or
  subformula1: STLCG.And
  subformula2: STLCG.LessThan
  operation: Maxish (function of type typeof(Maxish))


In [191]:
(f::STLCG.Formula)(x::Array{Float64}; kwargs...) = robustness_trace(f, x; kwargs...)

In [192]:
ϕ(x)

2×15×3 Array{Float64,3}:
[:, :, 1] =
 4.33878  4.55352  4.0096   4.28465  …  4.87325  4.24155  4.18938  4.36833
 4.35477  4.20984  4.53724  4.8834      4.25997  4.16101  4.62856  4.13871

[:, :, 2] =
 4.19292  4.05989  4.26052  4.3984   …  4.02068  4.74747  4.05824  4.06667
 4.13728  4.53258  4.77843  4.72391     4.18943  4.02867  4.20342  4.66529

[:, :, 3] =
 4.47263  4.89431  4.49562  4.28735  …  4.9606   4.3826   4.59063  4.70413
 4.29403  4.08497  4.01783  4.86847     4.10466  4.29218  4.78882  4.30469

In [147]:
struct Predicate
    x::Symbol
end

In [157]:
(p::Primitive)(x::Int) = (2*x, p.x)
(p::Primitive)(x::Real) = 3*x

In [158]:
a = Primitive(:asadsadsa)

Primitive(:asadsadsa)

In [159]:
a(1), a(1.)

((2, :asadsadsa), 3.0)

In [148]:
Primitive(:x) < 5

Primitive(:x)

In [172]:
Primitive(:a) > Primitive(:b)

false

In [170]:
(Base.:(<))(a::Symbol, b::Primitive) = a.x < b.x

In [75]:
function <(a,b)
    a < b
end

< (generic function with 1 method)

In [78]:
< |> methods

LoadError: syntax: "<" is not a unary operator